In [2]:
import pandas as pd
from glob import glob


In [3]:

# combine all raw daily CSV files into a single DataFrame
df = pd.concat((pd.read_csv(f) for f in glob('../data/raw/*')), ignore_index=True)
print(df.shape)
df.head()

(12192, 30)


,product_id,brand,gender,category,master_category,sub_category,article_type,mrp,price,discount,...,has_multiple_sizes,snapshot_date,season,is_fast_fashion,promotion_tags,has_promotion,year,preferred_delivery_tag,delivery_promise,source_sort
0,36716624,House of Sal,Women,Dresses,Apparel,Dress,Dresses,1990,1419,571,...,True,2026-01-13,Summer,True,NaN,False,2025,EXPRESS,Delivery By Jan 15,popularity
1,33810216,all about you,Women,Dresses,Apparel,Dress,Dresses,2999,779,2220,...,True,2026-01-13,Fall,True,NaN,False,2025,EXPRESS,Delivery By Jan 15,popularity
2,31157428,SANSKRUTIHOMES,Women,Dresses,Apparel,Dress,Dresses,2899,724,2175,...,True,2026-01-13,Spring,True,Crazy_Deal|Festive_Price_Crash|Myntra_Unique,True,2024,EXPRESS,Delivery By Jan 15,popularity
3,30082902,Bannos Swagger,Women,Dresses,Apparel,Dress,Dresses,4799,863,3936,...,True,2026-01-13,Fall,True,Crazy_Deal|GST_Benefit_Included|Festive_Price_...,True,2024,EXPRESS,Delivery By Jan 16,popularity
4,36266565,Phosphorus,Women,Dresses,Apparel,Dress,Dresses,2599,1039,1560,...,True,2026-01-13,Fall,True,NaN,False,2025,EXPRESS,Delivery By Jan 15,popularity


In [4]:
#combine all scraped data
df.to_csv('../data/combined_days.csv')
df.shape

(12192, 30)

In [5]:
#Checking product consistency across different scraping days 
df.groupby('product_id')['snapshot_date'].nunique().value_counts().sort_index()

snapshot_date
1    1274
2     493
3     440
4     868
5    1028
Name: count, dtype: int64

In [6]:
#create a feature indicating how mnay unique days each product appears
product_continuence = df.groupby('product_id')['snapshot_date'].nunique().reset_index(name='days_present')

df = df.merge(product_continuence, on='product_id', how='left')


In [7]:
#create continuity buckets
df['continuence_buckets'] = pd.cut(df['days_present'], bins=[0,1,2,3,5], labels = ['1_day', '2_day', '3_days', '4-5_days'])

In [8]:
df.columns

Index(['product_id', 'brand', 'gender', 'category', 'master_category',
       'sub_category', 'article_type', 'mrp', 'price', 'discount',
       'discount_type', 'discount_display_label', 'coupon_discount',
       'best_price', 'rating', 'rating_count', 'list_views', 'inventory',
       'available', 'sizes', 'has_multiple_sizes', 'snapshot_date', 'season',
       'is_fast_fashion', 'promotion_tags', 'has_promotion', 'year',
       'preferred_delivery_tag', 'delivery_promise', 'source_sort',
       'days_present', 'continuence_buckets'],
      dtype='object')

In [9]:
#products with no ratings are excluded
df_rated = df[df['rating_count']>0].copy()

df_rated['discount_pct'] = df_rated['discount']/df_rated['mrp']

#Keeping only products which are present for more 4-5 days 
stable = df_rated[df_rated['continuence_buckets'] == '4-5_days']
stable

,product_id,brand,gender,category,master_category,sub_category,article_type,mrp,price,discount,...,is_fast_fashion,promotion_tags,has_promotion,year,preferred_delivery_tag,delivery_promise,source_sort,days_present,continuence_buckets,discount_pct
1,33810216,all about you,Women,Dresses,Apparel,Dress,Dresses,2999,779,2220,...,True,NaN,False,2025,EXPRESS,Delivery By Jan 15,popularity,5,4-5_days,0.740247
3,30082902,Bannos Swagger,Women,Dresses,Apparel,Dress,Dresses,4799,863,3936,...,True,Crazy_Deal|GST_Benefit_Included|Festive_Price_...,True,2024,EXPRESS,Delivery By Jan 16,popularity,4,4-5_days,0.820171
4,36266565,Phosphorus,Women,Dresses,Apparel,Dress,Dresses,2599,1039,1560,...,True,NaN,False,2025,EXPRESS,Delivery By Jan 15,popularity,4,4-5_days,0.600231
8,29171632,Miss Chase,Women,Dresses,Apparel,Dress,Dresses,2999,1559,1440,...,True,Crazy_Deal|Festive_Price_Crash,True,2024,NaN,NaN,popularity,5,4-5_days,0.480160
14,25497076,W,Women,Dresses,Apparel,Dress,Dresses,4599,2299,2300,...,True,Crazy_Deal|Festive_Price_Crash,True,2023,EXPRESS,Delivery By Jan 15,popularity,4,4-5_days,0.500109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12187,32305216,Moda Rapido,Women,Dresses,Apparel,Dress,Dresses,2899,290,2609,...,True,Festive_Price_Crash,True,2025,EXPRESS,Delivery By Jan 16,discount,5,4-5_days,0.899966
12188,32305157,Moda Rapido,Women,Dresses,Apparel,Dress,Dresses,2899,290,2609,...,True,Festive_Price_Crash,True,2025,EXPRESS,Delivery By Jan 16,discount,5,4-5_days,0.899966
12189,16587816,plusS,Women,Dresses,Apparel,Dress,Dresses,2799,280,2519,...,True,NaN,False,2022,EXPRESS,Delivery By Jan 15,discount,4,4-5_days,0.899964
12190,16587814,plusS,Women,Dresses,Apparel,Dress,Dresses,2799,280,2519,...,True,NaN,False,2022,EXPRESS,Delivery By Jan 15,discount,4,4-5_days,0.899964


In [10]:
bins = [0, 0.4, 0.6, 0.75, 1]
labels = ["<40%", "40–60%", "60–75%", "75%+"]

stable['discount_bucket'] = pd.cut(stable['discount_pct'], bins=bins, labels=labels, include_lowest=True)

stable.head()

/tmp/ipykernel_2742/1769107142.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stable['discount_bucket'] = pd.cut(stable['discount_pct'], bins=bins, labels=labels, include_lowest=True)


,product_id,brand,gender,category,master_category,sub_category,article_type,mrp,price,discount,...,promotion_tags,has_promotion,year,preferred_delivery_tag,delivery_promise,source_sort,days_present,continuence_buckets,discount_pct,discount_bucket
1,33810216,all about you,Women,Dresses,Apparel,Dress,Dresses,2999,779,2220,...,NaN,False,2025,EXPRESS,Delivery By Jan 15,popularity,5,4-5_days,0.740247,60–75%
3,30082902,Bannos Swagger,Women,Dresses,Apparel,Dress,Dresses,4799,863,3936,...,Crazy_Deal|GST_Benefit_Included|Festive_Price_...,True,2024,EXPRESS,Delivery By Jan 16,popularity,4,4-5_days,0.820171,75%+
4,36266565,Phosphorus,Women,Dresses,Apparel,Dress,Dresses,2599,1039,1560,...,NaN,False,2025,EXPRESS,Delivery By Jan 15,popularity,4,4-5_days,0.600231,60–75%
8,29171632,Miss Chase,Women,Dresses,Apparel,Dress,Dresses,2999,1559,1440,...,Crazy_Deal|Festive_Price_Crash,True,2024,NaN,NaN,popularity,5,4-5_days,0.480160,40–60%
14,25497076,W,Women,Dresses,Apparel,Dress,Dresses,4599,2299,2300,...,Crazy_Deal|Festive_Price_Crash,True,2023,EXPRESS,Delivery By Jan 15,popularity,4,4-5_days,0.500109,40–60%


In [11]:
#calculate mean and median of ratings for discount buckets
rating_summary = stable.groupby('discount_bucket').agg( avg_rating = ('rating', 'mean'),
                                                        median_rating = ('rating', 'median'),
                                                        product_count = ('product_id', 'nunique'))

rating_summary

/tmp/ipykernel_2742/775126903.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_summary = stable.groupby('discount_bucket').agg( avg_rating = ('rating', 'mean'),


,avg_rating,median_rating,product_count
discount_bucket,,,
<40%,4.158541,4.229322,85
40–60%,4.149633,4.230690,262
60–75%,4.079683,4.188406,575
75%+,3.976022,4.080000,652


From abvoe results, we can say that
- For stable products, ratings decline gradually as discount depth increases, with the sharpest drop at over 75% discounts.
- Big discounts are associated with lower perceived quality.

In [12]:
import numpy as np

weighted = stable.groupby('discount_bucket').apply(lambda x: np.average(x['rating'], weights=x['rating_count'])).to_frame('weighted_avg_rating')

weighted

/tmp/ipykernel_2742/581967886.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  weighted = stable.groupby('discount_bucket').apply(lambda x: np.average(x['rating'], weights=x['rating_count'])).to_frame('weighted_avg_rating')
/tmp/ipykernel_2742/581967886.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted = stable.groupby('discount_bucket').apply(lambda x: np.average(x['rating'], weights=x['rating_count'])).to_frame('weighted_avg_rating')


,weighted_avg_rating
discount_bucket,
<40%,4.235412
40–60%,4.291799
60–75%,4.284004
75%+,4.186069


In [13]:
final_table = rating_summary.join(weighted)
print(final_table)

                 avg_rating  median_rating  product_count  weighted_avg_rating
discount_bucket                                                               
<40%               4.158541       4.229322             85             4.235412
40–60%             4.149633       4.230690            262             4.291799
60–75%             4.079683       4.188406            575             4.284004
75%+               3.976022       4.080000            652             4.186069


Weighted avg ratings are higher than avg rating, suggesting that products with more reviews have tend to have higher ratings.

In [14]:
stable['product_id'].nunique()

1517